## Initialize the project, set up the client and project

In [2]:
# Import the SDK and the client module
from label_studio_sdk import Client

LABEL_STUDIO_URL = 'https://app.heartex.com'
API_KEY = '<your-api-key>'

# Connect to the Label Studio API and check the connection
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

# Start new project 
project = ls.start_project(
    title='Storage Test Project',
    label_config='''
    <View>
      <Image name="image" value="$image" zoom="true"/>
      <PolygonLabels name="label" toName="image">
        <Label value="Airplane" background="red"/>
      </PolygonLabels>
    </View>
    '''
)

## Add a source (import) storage to the project

In [2]:
import_storage = project.connect_s3_import_storage(
    title="my storage", 
    bucket="htx-dev",
    prefix="dataset/training_set/dogs",
    aws_access_key_id="<s3-access-key>",
    aws_secret_access_key="<s3-secret-key>",
    use_blob_urls=True,  # "Treat every bucket object as a source file", more info: https://labelstud.io/guide/storage#Treat-every-bucket-object-as-a-source-file
    regex_filter=".*jpg"
)

In [3]:
import_storage

{'id': 17632,
 'type': 's3',
 'synchronizable': True,
 'presign': True,
 'last_sync': None,
 'last_sync_count': None,
 'last_sync_job': None,
 'status': 'initialized',
 'traceback': None,
 'meta': {},
 'title': 'my storage',
 'description': '',
 'created_at': '2024-03-12T02:00:05.404977Z',
 'bucket': 'htx-dev',
 'prefix': 'dataset/training_set/dogs',
 'regex_filter': '.*jpg',
 'use_blob_urls': True,
 'aws_session_token': None,
 'aws_sse_kms_key_id': None,
 'region_name': None,
 's3_endpoint': None,
 'presign_ttl': 1,
 'recursive_scan': False,
 'project': 58689}

## Sync import storage to get tasks from the storage 

In [4]:
sync_result = project.sync_import_storage(import_storage['type'], import_storage['id'])

In [5]:
sync_result

{'id': 17632,
 'type': 's3',
 'synchronizable': True,
 'presign': True,
 'last_sync': None,
 'last_sync_count': None,
 'last_sync_job': 'f8648608-fbb5-4590-a660-9631803fd95b',
 'status': 'queued',
 'traceback': None,
 'meta': {'attempts': 1, 'time_queued': '2024-03-12 02:00:06.426879+00:00'},
 'title': 'my storage',
 'description': '',
 'created_at': '2024-03-12T02:00:05.404977Z',
 'bucket': 'htx-dev',
 'prefix': 'dataset/training_set/dogs',
 'regex_filter': '.*jpg',
 'use_blob_urls': True,
 'aws_session_token': None,
 'aws_sse_kms_key_id': None,
 'region_name': None,
 's3_endpoint': None,
 'presign_ttl': 1,
 'recursive_scan': False,
 'project': 58689}

## Get status of synchronization for the storage

And wait until sync is completed. 

In [6]:
import time

while import_storage['status'] != 'completed':
    time.sleep(3.0)  # wait for 3 seconds    
    
    # get all import storages and find the one we need by id
    import_storages = project.get_import_storages()
    for storage in import_storages:
        if import_storage['id'] == storage['id']:
            import_storage = storage
            print(storage['id'], storage['title'], storage['status'])

print(f'Import storage {import_storage["id"]} is synced!')

17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage in_progress
17632 my storage completed
Import storage 17632 is synced!


## Add a target (export) storage to the project

In [4]:
export_storage = project.connect_s3_export_storage(
    title="my export storage", 
    bucket="htx-dev",
    prefix="test_out",
    aws_access_key_id="<s3-access-key>",
    aws_secret_access_key="<s3-secret-key>",
    can_delete_objects=False  # if you delete annotations in Label Studio, the corresponding objects in the storage will not be deleted
)

In [5]:
export_storage

{'id': 2975,
 'type': 's3',
 'synchronizable': True,
 'last_sync': None,
 'last_sync_count': None,
 'last_sync_job': 'd73a8c0d-a600-47f7-b36c-2a0f112a937f',
 'status': 'queued',
 'traceback': None,
 'meta': {'attempts': 1, 'time_queued': '2024-03-12 02:06:30.977353+00:00'},
 'title': 'my export storage',
 'description': '',
 'created_at': '2024-03-12T02:06:30.971049Z',
 'can_delete_objects': False,
 'bucket': 'htx-dev',
 'prefix': 'test_out',
 'regex_filter': None,
 'use_blob_urls': False,
 'aws_session_token': None,
 'aws_sse_kms_key_id': None,
 'region_name': None,
 's3_endpoint': None,
 'project': 58690}

## Sync target storage to save annotations to the storage

Target storages sync automatically, so we don't need to call any methods to sync them. However, if you need to re-sync the storage from the scratch, you can call `sync_export_storage` method.

In [6]:
sync_result = project.sync_export_storage(export_storage['type'], export_storage['id'])
sync_result

{'id': 2975,
 'type': 's3',
 'synchronizable': True,
 'last_sync': None,
 'last_sync_count': None,
 'last_sync_job': 'de6f4d81-3085-49b8-87ef-8ed63fbe1e62',
 'status': 'queued',
 'traceback': None,
 'meta': {'attempts': 2, 'time_queued': '2024-03-12 02:06:34.927315+00:00'},
 'title': 'my export storage',
 'description': '',
 'created_at': '2024-03-12T02:06:30.971049Z',
 'can_delete_objects': False,
 'bucket': 'htx-dev',
 'prefix': 'test_out',
 'regex_filter': None,
 'use_blob_urls': False,
 'aws_session_token': None,
 'aws_sse_kms_key_id': None,
 'region_name': None,
 's3_endpoint': None,
 'project': 58690}

## Get status of synchronization for the target storage

And wait until sync is completed.

In [7]:
import time

while export_storage['status'] != 'completed':
    time.sleep(3.0)  # wait for 3 seconds    
    
    # get all import storages and find the one we need by id
    export_storages = project.get_export_storages()
    for storage in export_storages:
        if export_storage['id'] == storage['id']:
            export_storage = storage
            print(storage['id'], storage['title'], storage['status'])

print(f'Export storage {export_storage["id"]} is synced!')

2975 my export storage completed
Export storage 2975 is synced!


In [8]:
export_storage

{'id': 2975,
 'type': 's3',
 'synchronizable': True,
 'last_sync': '2024-03-12T02:06:34.986049Z',
 'last_sync_count': 0,
 'last_sync_job': 'de6f4d81-3085-49b8-87ef-8ed63fbe1e62',
 'status': 'completed',
 'traceback': None,
 'meta': {'attempts': 2,
  'duration': 0.005824,
  'time_queued': '2024-03-12 02:06:34.927315+00:00',
  'time_completed': '2024-03-12 02:06:34.986054+00:00',
  'time_last_ping': '2024-03-12 02:06:34.980230+00:00',
  'time_in_progress': '2024-03-12 02:06:34.980230+00:00',
  'total_annotations': 0},
 'title': 'my export storage',
 'description': '',
 'created_at': '2024-03-12T02:06:30.971049Z',
 'can_delete_objects': False,
 'bucket': 'htx-dev',
 'prefix': 'test_out',
 'regex_filter': None,
 'use_blob_urls': False,
 'aws_session_token': None,
 'aws_sse_kms_key_id': None,
 'region_name': None,
 's3_endpoint': None,
 'project': 58690}